In [1]:
library(RPostgreSQL)
library(twang)
library(Matching)
library(tidyverse)
library(survey)

Warning message:
"package 'RPostgreSQL' was built under R version 4.3.3"
Loading required package: DBI



Warning message:
"package 'twang' was built under R version 4.3.3"
Warning message in .recacheSubclasses(def@className, def, env):
"undefined subclass "ndiMatrix" of class "replValueSp"; definition not updated"
To reproduce results from prior versions of the twang package, please see the version="legacy" option described in the documentation.

Warning message:
"package 'Matching' was built under R version 4.3.3"
Loading required package: MASS

## 
##  Matching (Version 4.10-14, Build Date: 2023-09-13)
##  See https://www.jsekhon.com for additional documentation.
##  Please cite software as:
##   Jasjeet S. Sekhon. 2011. ``Multivariate and Propensity Score Matching
##   Software with Automated Balance Optimization: The Matching package for R.''
##   Journal of Statistical Software, 42(7): 1-52. 
##


Warning message:
"package 'tidyverse' was built under R version 4.3.3"
Warning message:
"package 'ggplot2' was built under R version 4.3.3"
Warning message:
"package 'readr' was built under

In [2]:
data_dir <- file.path("..", "data")
sql_dir <- file.path("..", "sql")

In [3]:

con <- dbConnect(RPostgres::Postgres(),
                  host = "127.0.0.1",
                  port = 5432,
                  dbname = "mimiciv",
                  user = "postgres",
                  password = "123")


In [4]:
full_data = dbGetQuery(con, "select*from mimiciv_hosp.merged_data_gcs")
head(full_data)
colnames(full_data)


,stay_id,hadm_id,subject_id,gender,age,sapsii,icu_intime,icu_outtime,weight,sofa_24hours,⋯,lab_sodium_flag,lab_bun_flag,lab_bicarbonate_flag,lab_bnp_flag,lab_pco2_flag,lab_creatinine_flag,lab_potassium_flag,lab_troponin_flag,lab_po2_flag,lab_lactate_flag
,<int>,<int>,<int>,<chr>,<dbl>,<int>,<dttm>,<dttm>,<dbl>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,38302142,20475690,13620446,F,66.04835,31,2159-01-21 01:45:32,2159-01-25 14:09:02,138.0,1,⋯,1,1,1,0,1,1,1,0,1,1
2,31869153,21383344,15359363,F,41.91122,39,2166-12-02 16:08:45,2166-12-05 02:10:02,70.0,0,⋯,1,1,1,0,0,1,1,0,0,0
3,31501167,20944291,12162143,F,87.59137,35,2174-08-05 00:49:12,2174-08-06 16:13:22,65.0,1,⋯,1,1,1,0,1,1,1,0,1,1
4,39256124,24959920,18226317,M,66.88656,28,2130-11-20 19:28:29,2130-11-25 12:42:59,88.0,3,⋯,1,1,1,0,0,1,1,0,0,0
5,31132695,28164203,15928453,F,44.39155,55,2113-05-24 00:14:32,2113-05-29 14:24:45,45.9,10,⋯,1,1,1,0,1,1,1,0,1,1
6,39784690,23246205,16810503,M,56.61848,30,2149-08-14 21:32:37,2149-08-21 12:15:14,80.0,3,⋯,1,1,1,0,1,1,1,0,1,1


[1] "stay_id"                        "hadm_id"                       
  [3] "subject_id"                     "gender"                        
  [5] "age"                            "sapsii"                        
  [7] "icu_intime"                     "icu_outtime"                   
  [9] "weight"                         "sofa_24hours"                  
 [11] "infection"                      "mort_28_day"                   
 [13] "survival_days"                  "gcs_include"                   
 [15] "gcs_include_combine"            "vs_heart_rate_first"           
 [17] "vs_heart_rate_min"              "vs_heart_rate_max"             
 [19] "vs_cvp_first"                   "vs_cvp_min"                    
 [21] "vs_cvp_max"                     "vs_map_first"                  
 [23] "vs_map_min"                     "vs_map_max"                    
 [25] "vs_temp_first"                  "vs_temp_min"                   
 [27] "vs_temp_max"                    "lab_hemoglobin_first"          
 [29] "lab_hemoglobin_min"             "lab_hemoglobin_max"            
 [31] "lab_hemoglobin_abnormal"        "lab_platelet_first"            
 [33] "lab_platelet_min"               "lab_platelet_max"              
 [35] "lab_platelet_abnormal"          "lab_creatinine_kinase_first"   
 [37] "lab_creatinine_kinase_min"      "lab_creatinine_kinase_max"     
 [39] "lab_creatinine_kinase_abnormal" "lab_wbc_first"                 
 [41] "lab_wbc_min"                    "lab_wbc_max"                   
 [43] "lab_wbc_abnormal"               "lab_ph_first"                  
 [45] "lab_ph_min"                     "lab_ph_max"                    
 [47] "lab_ph_abnormal"                "lab_chloride_first"            
 [49] "lab_chloride_min"               "lab_chloride_max"              
 [51] "lab_chloride_abnormal"          "lab_sodium_first"              
 [53] "lab_sodium_min"                 "lab_sodium_max"                
 [55] "lab_sodium_abnormal"            "lab_bun_first"                 
 [57] "lab_bun_min"                    "lab_bun_max"                   
 [59] "lab_bun_abnormal"               "lab_bicarbonate_first"         
 [61] "lab_bicarbonate_min"            "lab_bicarbonate_max"           
 [63] "lab_bicarbonate_abnormal"       "lab_pco2_first"                
 [65] "lab_pco2_min"                   "lab_pco2_max"                  
 [67] "lab_pco2_abnormal"              "lab_creatinine_first"          
 [69] "lab_creatinine_min"             "lab_creatinine_max"            
 [71] "lab_creatinine_abnormal"        "lab_potassium_first"           
 [73] "lab_potassium_min"              "lab_potassium_max"             
 [75] "lab_potassium_abnormal"         "lab_troponin_first"            
 [77] "lab_troponin_min"               "lab_troponin_max"              
 [79] "lab_troponin_abnormal"          "lab_po2_first"                 
 [81] "lab_po2_min"                    "lab_po2_max"                   
 [83] "lab_po2_abnormal"               "lab_lactate_first"             
 [85] "lab_lactate_min"                "lab_lactate_max"               
 [87] "lab_lactate_abnormal"           "pre_dose_val_rx"               
 [89] "los_drug"                       "cumulative_dose"               
 [91] "gcs"                            "ca"                            
 [93] "infection_flag"                 "dose_val_rx_average"           
 [95] "los_drug_average"               "event_state"                   
 [97] "vs_heart_rate_flag"             "vs_cvp_flag"                   
 [99] "vs_map_flag"                    "vs_temp_flag"                  
[101] "lab_hemoglobin_flag"            "lab_platelet_flag"             
[103] "lab_creatinine_kinase_flag"     "lab_wbc_flag"                  
[105] "lab_ph_flag"                    "lab_chloride_flag"             
[107] "lab_sodium_flag"                "lab_bun_flag"                  
[109] "lab_bicarbonate_flag"           "lab_bnp_flag"                  
[111] "lab_pco2_flag"                  "la

In [5]:
to_factor <- function(x) {
    if(length(na.omit(unique(x))) <= 1) return(factor(x, levels = c(0, 1)))
    return(factor(x))
}

In [6]:

factor_vars <- full_data %>%
    names %>%
    grep("flag|abnormal|gcs", ., value = TRUE) %>%
    c("gender",  "mort_28_day", "event_state", "gcs", "ca","infection")
factor_vars

[1] "gcs_include"                    "gcs_include_combine"           
 [3] "lab_hemoglobin_abnormal"        "lab_platelet_abnormal"         
 [5] "lab_creatinine_kinase_abnormal" "lab_wbc_abnormal"              
 [7] "lab_ph_abnormal"                "lab_chloride_abnormal"         
 [9] "lab_sodium_abnormal"            "lab_bun_abnormal"              
[11] "lab_bicarbonate_abnormal"       "lab_pco2_abnormal"             
[13] "lab_creatinine_abnormal"        "lab_potassium_abnormal"        
[15] "lab_troponin_abnormal"          "lab_po2_abnormal"              
[17] "lab_lactate_abnormal"           "gcs"                           
[19] "infection_flag"                 "vs_heart_rate_flag"            
[21] "vs_cvp_flag"                    "vs_map_flag"                   
[23] "vs_temp_flag"                   "lab_hemoglobin_flag"           
[25] "lab_platelet_flag"              "lab_creatinine_kinase_flag"    
[27] "lab_wbc_flag"                   "lab_ph_flag"                   
[29] "lab_chloride_flag"              "lab_sodium_flag"               
[31] "lab_bun_flag"                   "lab_bicarbonate_flag"          
[33] "lab_bnp_flag"                   "lab_pco2_flag"                 
[35] "lab_creatinine_flag"            "lab_potassium_flag"            
[37] "lab_troponin_flag"              "lab_po2_flag"                  
[39] "lab_lactate_flag"               "gender"                        
[41] "mort_28_day"                    "event_state"                   
[43] "gcs"                            "ca"                            
[45] "infection"

In [7]:
full_data <- full_data %>%
  mutate(gcs_int = as.integer(gcs)) %>%
  mutate(ca_int = as.integer(ca)) %>%
  mutate(event_state_int = as.integer(event_state)) %>%
  mutate(mort_28_day_int = as.integer(mort_28_day)) %>%
  mutate_at(factor_vars, to_factor) %>%
  mutate(gender = relevel(gender, "M")) 



In [8]:
feature_names <- full_data %>%
    names %>%
    keep(grepl("vs|lab|age|gender|weight|sapsii|sofa|sofa_24hours|event_state|cam_state|gcs", .)) %>%
    discard(grepl("vs|lab", .) & grepl("flag", .) & !grepl("bnp|troponin|kinase|cvp", .)) %>%

    discard(grepl("min|max|flag|gcs_include", .)) %>%
    discard(grepl("abnormal", .))
feature_names
length(feature_names)

[1] "gender"                      "age"                        
 [3] "sapsii"                      "weight"                     
 [5] "sofa_24hours"                "vs_heart_rate_first"        
 [7] "vs_cvp_first"                "vs_map_first"               
 [9] "vs_temp_first"               "lab_hemoglobin_first"       
[11] "lab_platelet_first"          "lab_creatinine_kinase_first"
[13] "lab_wbc_first"               "lab_ph_first"               
[15] "lab_chloride_first"          "lab_sodium_first"           
[17] "lab_bun_first"               "lab_bicarbonate_first"      
[19] "lab_pco2_first"              "lab_creatinine_first"       
[21] "lab_potassium_first"         "lab_troponin_first"         
[23] "lab_po2_first"               "lab_lactate_first"          
[25] "gcs"                         "dose_val_rx_average"        
[27] "los_drug_average"            "event_state"                
[29] "gcs_int"                     "event_state_int"

[1] 30

In [9]:

feature_names %>%
    data.frame(feature = ., stringsAsFactors = FALSE) %>%
    data.table::fwrite(file = file.path(data_dir, "feature_names2.csv"))




In [10]:
features <- full_data %>%
    dplyr::select(!!!rlang::syms(feature_names))
head(features)






,gender,age,sapsii,weight,sofa_24hours,vs_heart_rate_first,vs_cvp_first,vs_map_first,vs_temp_first,lab_hemoglobin_first,⋯,lab_potassium_first,lab_troponin_first,lab_po2_first,lab_lactate_first,gcs,dose_val_rx_average,los_drug_average,event_state,gcs_int,event_state_int
,<fct>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<chr>,<dbl>,<fct>,<int>,<int>
1,F,66.04835,31,138.0,1,143,NA,78,38.27778,8.5,⋯,3.9,NA,40,1.1,0,NA,NA,1,0,1
2,F,41.91122,39,70.0,0,140,NA,99,39.44444,8.0,⋯,3.0,NA,NA,NA,1,50,2,0,1,0
3,F,87.59137,35,65.0,1,97,NA,110,36.72222,11.4,⋯,3.7,NA,40,2.7,0,5,136,0,0,0
4,M,66.88656,28,88.0,3,72,NA,82,37.05556,13.5,⋯,4.1,NA,NA,NA,0,NA,NA,0,0,0
5,F,44.39155,55,45.9,10,110,16,78,37.33333,13.0,⋯,2.5,NA,296,2.4,0,NA,NA,1,0,1
6,M,56.61848,30,80.0,3,110,4,86,NA,11.4,⋯,4.3,NA,223,1.0,0,NA,NA,0,0,0


In [11]:
label_name <- "gcs"
label <- full_data %>% pull(gcs)
str(label)

 Factor w/ 2 levels "0","1": 1 2 1 1 1 1 2 1 1 1 ...


In [12]:
label <- full_data %>% pull(gcs)
str(label)

 Factor w/ 2 levels "0","1": 1 2 1 1 1 1 2 1 1 1 ...


In [13]:
 full_data$pre_dose_val_rx <- as.numeric(full_data$pre_dose_val_rx)

In [14]:

fml <- "mort_28_day ~ gcs + gender + age + sapsii + weight + sofa_24hours + vs_heart_rate_first + vs_map_first + vs_temp_first + lab_hemoglobin_first + lab_platelet_first + lab_wbc_first + lab_ph_first + lab_chloride_first + lab_sodium_first + lab_bun_first + lab_bicarbonate_first + lab_pco2_first + lab_creatinine_first + lab_potassium_first + lab_po2_first + lab_lactate_first"
fml


[1] "mort_28_day ~ gcs + gender + age + sapsii + weight + sofa_24hours + vs_heart_rate_first + vs_map_first + vs_temp_first + lab_hemoglobin_first + lab_platelet_first + lab_wbc_first + lab_ph_first + lab_chloride_first + lab_sodium_first + lab_bun_first + lab_bicarbonate_first + lab_pco2_first + lab_creatinine_first + lab_potassium_first + lab_po2_first + lab_lactate_first"

fml <- feature_names %>%
    c("gcs", .) %>%
    paste(collapse = " + ") %>%
    sprintf("mort_28_day ~ %s", .)
fml

In [15]:
unweighted <- glm(as.formula(fml), data = full_data, family = binomial, na.action = na.exclude)
summary(unweighted)


exp(cbind(OR = coef(unweighted), confint(unweighted)))

## 这里得到的有效变量非常多，而且主要分类变量的意义比较大，这是通过回归之后得到的结果，所以在进行分析的第一步就可以用回归的方法来研究有效变量。



Call:
glm(formula = as.formula(fml), family = binomial, data = full_data, 
    na.action = na.exclude)

Coefficients:
                        Estimate Std. Error z value Pr(>|z|)    
(Intercept)           -4.9834123  9.8553248  -0.506 0.613098    
gcs1                  -0.0404799  0.2292858  -0.177 0.859864    
genderF                0.3942072  0.1631194   2.417 0.015663 *  
age                    0.0208507  0.0061461   3.393 0.000693 ***
sapsii                 0.0330349  0.0061967   5.331 9.76e-08 ***
weight                -0.0137895  0.0039460  -3.495 0.000475 ***
sofa_24hours           0.2162730  0.0247158   8.750  < 2e-16 ***
vs_heart_rate_first    0.0074115  0.0037301   1.987 0.046928 *  
vs_map_first          -0.0002146  0.0038383  -0.056 0.955418    
vs_temp_first         -0.1044461  0.0807706  -1.293 0.195969    
lab_hemoglobin_first  -0.0386125  0.0372900  -1.035 0.300452    
lab_platelet_first     0.0004273  0.0006381   0.670 0.503116    
lab_wbc_first          0.0031915  0.

Waiting for profiling to be done...



,OR,2.5 %,97.5 %
(Intercept),0.006850646,1.921037e-11,1.173442e+06
gcs1,0.960328470,6.048227e-01,1.488672e+00
genderF,1.483207909,1.078862e+00,2.046237e+00
age,1.021069616,1.008956e+00,1.033579e+00
sapsii,1.033586655,1.021089e+00,1.046220e+00
weight,0.986305111,9.785687e-01,9.938306e-01
sofa_24hours,1.241441265,1.183413e+00,1.303950e+00
vs_heart_rate_first,1.007439033,1.000089e+00,1.014833e+00
vs_map_first,0.999785443,9.922002e-01,1.007236e+00
vs_temp_first,0.900823340,7.710793e-01,1.057658e+00


In [16]:
saveRDS(unweighted, file = file.path(data_dir, "multivariate_model.rds"))

In [17]:
# 计算结果并存储在变量中
result <- exp(cbind(OR = coef(unweighted), confint(unweighted)))

# 将协变量的名称添加为新的一列
result <- data.frame(Variable = rownames(result), result)

# 将结果写入 CSV 文件
data.table::fwrite(result, file.path(data_dir, "result.csv"))

Waiting for profiling to be done...



In [18]:
fml <-  "gcs_int ~ gender + age + sapsii + weight + sofa_24hours + vs_heart_rate_first + vs_map_first + vs_temp_first + lab_hemoglobin_first + lab_platelet_first + lab_wbc_first + lab_ph_first + lab_chloride_first + lab_sodium_first + lab_bun_first + lab_bicarbonate_first +  lab_creatinine_first + lab_potassium_first + lab_po2_flag + lab_pco2_flag + lab_lactate_flag  + lab_creatinine_kinase_flag +lab_bnp_flag +infection_flag"



In [19]:
echo_ps_ate <- ps(as.formula(fml),
                  data = full_data,
                  interaction.depth = 2,
                  shrinkage = 0.01,
                  perm.test.iters = 0,
                  estimand = "ATE",
                  verbose = FALSE,
                  stop.method = c("es.mean", "es.max", "ks.mean", "ks.max"),
                  n.trees = 10000,
                  train.fraction = 0.8, 
                  cv.folds = 3, 
                  early_stopping_rounds = 20,
                  version = "legacy",
                  subsample = 0.3,
                  colsample_bytree = 0.3,  
                  n.cores = 8)

In [20]:
library("survey")

pred <- echo_ps_ate$ps$es.mean.ATE
full_data <- full_data %>% mutate(ps = pred)
ROCR::performance(ROCR::prediction(pred, label), "auc")@y.values %>% first



[1] 0.80543

In [21]:
ft_importance <- summary(echo_ps_ate$gbm.obj,
                         n.trees = echo_ps_ate$desc$es.mean.ATE$n.trees,
                         plot = FALSE)
ft_importance

,var,rel.inf
,<chr>,<dbl>
lab_wbc_first,lab_wbc_first,12.06265284
lab_platelet_first,lab_platelet_first,11.60184076
weight,weight,8.47860015
vs_heart_rate_first,vs_heart_rate_first,7.71049272
lab_hemoglobin_first,lab_hemoglobin_first,7.51525036
vs_temp_first,vs_temp_first,6.24292049
lab_creatinine_first,lab_creatinine_first,5.90376144
lab_bun_first,lab_bun_first,5.37278246
vs_map_first,vs_map_first,5.19796557


In [22]:
full_data <- full_data %>%
    mutate(ps_weight = get.weights(echo_ps_ate, stop.method = "es.mean"))

In [23]:
saveRDS(full_data, file = file.path(data_dir, "full_data_ps.rds"))

In [24]:
saveRDS(ft_importance, file = file.path(data_dir, "feature_importance.rds"))

In [25]:

primary_ipw <- glm(mort_28_day ~ gcs, data = full_data,
                   weights = full_data$ps_weight, family = binomial)
summary(primary_ipw)
exp(cbind(OR = coef(primary_ipw), confint(primary_ipw)))



Warning message in eval(family$initialize):
"non-integer #successes in a binomial glm!"



Call:
glm(formula = mort_28_day ~ gcs, family = binomial, data = full_data, 
    weights = full_data$ps_weight)

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -2.20096    0.05058 -43.515   <2e-16 ***
gcs1        -0.09500    0.07781  -1.221    0.222    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 4932.8  on 4400  degrees of freedom
Residual deviance: 4931.3  on 4399  degrees of freedom
AIC: 4618.3

Number of Fisher Scoring iterations: 5


Waiting for profiling to be done...

Warning message in eval(family$initialize):
"non-integer #successes in a binomial glm!"
Warning message in eval(family$initialize):
"non-integer #successes in a binomial glm!"
Warning message in eval(family$initialize):
"non-integer #successes in a binomial glm!"
Warning message in eval(family$initialize):
"non-integer #successes in a binomial glm!"
Warning message in eval(family$initialize):
"non-integer #successes in a binomial glm!"
Warning message in eval(family$initialize):
"non-integer #successes in a binomial glm!"
Warning message in eval(family$initialize):
"non-integer #successes in a binomial glm!"
Warning message in eval(family$initialize):
"non-integer #successes in a binomial glm!"
Warning message in eval(family$initialize):
"non-integer #successes in a binomial glm!"
Warning message in eval(family$initialize):
"non-integer #successes in a binomial glm!"
Warning message in eval(family$initialize):
"non-integer #successes in a binomial g

,OR,2.5 %,97.5 %
(Intercept),0.1106966,0.1001150,0.1220753
gcs1,0.9093735,0.7803023,1.0587280


In [26]:
saveRDS(primary_ipw, file = file.path(data_dir, "ipw_model.rds"))

In [27]:

ipw_svydesign <- svydesign(ids = ~ stay_id, weights = ~ ps_weight, data = full_data)


In [28]:
saveRDS(ipw_svydesign, file = file.path(data_dir, "ipw_svydesign.rds"))

In [29]:
fml

[1] "gcs_int ~ gender + age + sapsii + weight + sofa_24hours + vs_heart_rate_first + vs_map_first + vs_temp_first + lab_hemoglobin_first + lab_platelet_first + lab_wbc_first + lab_ph_first + lab_chloride_first + lab_sodium_first + lab_bun_first + lab_bicarbonate_first +  lab_creatinine_first + lab_potassium_first + lab_po2_flag + lab_pco2_flag + lab_lactate_flag  + lab_creatinine_kinase_flag +lab_bnp_flag +infection_flag"

In [30]:
str(full_data)

'data.frame':	4401 obs. of  122 variables:
 $ stay_id                       : int  38302142 31869153 31501167 39256124 31132695 39784690 30676468 36764451 37622123 33677073 ...
 $ hadm_id                       : int  20475690 21383344 20944291 24959920 28164203 23246205 25726133 25762316 23409728 25424583 ...
 $ subject_id                    : int  13620446 15359363 12162143 18226317 15928453 16810503 18111896 15558620 13263395 12252649 ...
 $ gender                        : Factor w/ 2 levels "M","F": 2 2 2 1 2 1 1 2 1 1 ...
 $ age                           : num  66 41.9 87.6 66.9 44.4 ...
 $ sapsii                        : int  31 39 35 28 55 30 62 35 20 46 ...
 $ icu_intime                    : POSIXct, format: "2159-01-21 01:45:32" "2166-12-02 16:08:45" ...
 $ icu_outtime                   : POSIXct, format: "2159-01-25 14:09:02" "2166-12-05 02:10:02" ...
 $ weight                        : num  138 70 65 88 45.9 80 99.7 72 89.6 90.5 ...
 $ sofa_24hours                  : int  1 0 

'mort_28_day ~ gender + age + sapsii + weight + sofa_24hours + vs_heart_rate_first + vs_cvp_first + vs_map_first + vs_temp_first + lab_hemoglobin_first + lab_platelet_first + lab_wbc_first + lab_ph_first + lab_chloride_first + lab_sodium_first + lab_bun_first + lab_bicarbonate_first + lab_pco2_first + lab_creatinine_first + lab_potassium_first + lab_troponin_flag + lab_po2_first + lab_lactate_first  + vs_cvp_flag + + lab_creatinine_kinase_flag +lab_bnp_flag + infection_flag'

In [31]:
fml <- feature_names %>%
    c(label_name, .) %>%
    paste(collapse = " + ") %>%
    sprintf("mort_28_day ~ %s", .)
fml

[1] "mort_28_day ~ gcs + gender + age + sapsii + weight + sofa_24hours + vs_heart_rate_first + vs_cvp_first + vs_map_first + vs_temp_first + lab_hemoglobin_first + lab_platelet_first + lab_creatinine_kinase_first + lab_wbc_first + lab_ph_first + lab_chloride_first + lab_sodium_first + lab_bun_first + lab_bicarbonate_first + lab_pco2_first + lab_creatinine_first + lab_potassium_first + lab_troponin_first + lab_po2_first + lab_lactate_first + gcs + dose_val_rx_average + los_drug_average + event_state + gcs_int + event_state_int"

In [32]:
fml <-  "mort_28_day ~ gcs + gender + age + sapsii + weight + sofa_24hours + infection_flag + vs_heart_rate_first + vs_map_first + vs_temp_first + lab_hemoglobin_first + lab_platelet_first  + lab_wbc_first + lab_ph_first + lab_chloride_first + lab_sodium_first + lab_bun_first + lab_bicarbonate_first + lab_pco2_first + lab_creatinine_first + lab_potassium_first+ lab_po2_first + lab_lactate_first + lab_creatinine_kinase_flag + lab_bnp_flag"
fml


[1] "mort_28_day ~ gcs + gender + age + sapsii + weight + sofa_24hours + infection_flag + vs_heart_rate_first + vs_map_first + vs_temp_first + lab_hemoglobin_first + lab_platelet_first  + lab_wbc_first + lab_ph_first + lab_chloride_first + lab_sodium_first + lab_bun_first + lab_bicarbonate_first + lab_pco2_first + lab_creatinine_first + lab_potassium_first+ lab_po2_first + lab_lactate_first + lab_creatinine_kinase_flag + lab_bnp_flag"

In [33]:

logi <- svyglm(as.formula(fml),
               family = quasibinomial,
               design = ipw_svydesign)


fml <- feature_names %>%
    c(label_name, .) %>%
    paste(collapse = " + ") %>%
    sprintf("mort_28_day ~ %s", .)
fml

In [34]:
summary(logi)


Call:
svyglm(formula = as.formula(fml), design = ipw_svydesign, family = quasibinomial)

Survey design:
svydesign(ids = ~stay_id, weights = ~ps_weight, data = full_data)

Coefficients:
                              Estimate Std. Error t value Pr(>|t|)    
(Intercept)                  6.8643512 15.5172676   0.442 0.658267    
gcs1                        -0.1731024  0.2438105  -0.710 0.477789    
genderF                      0.3711136  0.2421241   1.533 0.125487    
age                          0.0301495  0.0087548   3.444 0.000585 ***
sapsii                       0.0303557  0.0089286   3.400 0.000687 ***
weight                      -0.0106673  0.0053413  -1.997 0.045937 *  
sofa_24hours                 0.2222548  0.0446558   4.977 6.97e-07 ***
infection_flag1              0.9908395  0.3388030   2.925 0.003486 ** 
vs_heart_rate_first          0.0132728  0.0061509   2.158 0.031048 *  
vs_map_first                 0.0044783  0.0053379   0.839 0.401578    
vs_temp_first               -0.02

In [35]:
exp(cbind(OR = coef(logi), confint(logi)))

,OR,2.5 %,97.5 %
(Intercept),957.5243771,5.826093e-11,1.573701e+16
gcs1,0.8410515,5.214031e-01,1.356662e+00
genderF,1.4493478,9.014880e-01,2.330157e+00
age,1.0306086,1.013065e+00,1.048456e+00
sapsii,1.0308211,1.012929e+00,1.049029e+00
weight,0.9893894,9.790800e-01,9.998074e-01
sofa_24hours,1.2488896,1.144173e+00,1.363191e+00
infection_flag1,2.6934946,1.386002e+00,5.234418e+00
vs_heart_rate_first,1.0133612,1.001211e+00,1.025659e+00
vs_map_first,1.0044884,9.940283e-01,1.015059e+00


In [36]:
saveRDS(logi, file = file.path(data_dir, "doubly_robust_all_model.rds"))

In [37]:

result2 <- exp(cbind(OR = coef(logi), confint(logi)))
result2 <- data.frame(Variable = rownames(result2), result2)
data.table::fwrite(result2, file.path(data_dir, "result2.csv"))